<a href="https://www.bigdatauniversity.com"><img src = "https://ibm.box.com/shared/static/cw2c7r3o20w9zn8gkecaeyjhgw3xdgbj.png" width = 400, align = "center"></a>


<h1 align=center><font size = 5>K-Nearest Neighbors in R</font></h1>

In this lesson, we are going to introduce the $K$-nearest neighbors (KNN) algorithm and show some practical ways of using it in `R` with the `knn` function that exists in the `class` library. Later, we will show how to tune it with the `caret` library.

For a simple binary classification task (two class classification, $A$, $B$), *given* training and testing datasets and a positive integer $K$, for **each record in the test dataset**, KNN tries to find $K$ neighbors in **training set** that are *closest* to that test record and *counts* how many of those $K$ examples in the training set belong to class $A,$ and how many belong to class $B.$ The test record is then classified as belonging to the majority class (based on counted votes) i.e. the test record is considered to be of class $i$ if the majority of the $K$-nearest neighbors in the training set belong to class $i.$

As can be seen, there are no parameters that *need to be learned during training* to determine whether a new observation belongs to class $A$ or $B.$ The only parameter used in k-nearest neighbors is k, which is a predetermined value. The algorithm simply works by looking at the training samples, calculating *distances* and finding the $K$ examples in the training set that are closest to the new observation. Thus, KNN is a *non-parametric,* supervised (needs training labels) learning algorithm.

The following diagram illustrates the main idea of how the k-nearest neighbors algorithm works. As $K$ varies from $3$ to $6$ the class of the new observation (red star) changes from $B$ to $A$ because the majority votes are changed. That is, for $K=3,$ we have two observations of class $B$ and one of class $A$, while for $K=6,$ we have two observations of class $B,$ and four of class $A.$

<img src="http://bdewilde.github.io/assets/images/2012-10-26-knn-concept.png" width=600, align = "center">
<div style="text-align:center"> [[KNN classifications for k=3 and k=6]](http://bdewilde.github.io/blog/blogger/2012/10/26/classification-of-hand-written-digits-3/)
<br>

For a regression task, the same method can be applied but instead of taking majority votes, we can, for example, find the mean of the $response$ $variable$ of the $K$-nearest neighbors from the new observation.

KNN depends on 1) the choice of metric (for example, Euclidean in above example), and 2) the choice of $K.$ There are no universal choices, and depending on the data, one has to examine various options to find a suitable choice.

*Caveats:*

* When using KNN, we must ensure that there are no categorical variables (factors) involved in the **features**, simply because one cannot find the distance from them. For example, when a categorical variable takes values from the set {apple, orange, banana, grapes ...}, one cannot make use of numerical distance functions, unless of course there is a pre-determined way to evaluate these distance from a qualitative standpoint.

* If the training set is high-dimensional, KNN will suffer from [the curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality). Therefore, we could use a dimensionality reduction technique prior to using KNN.

* *Standardize* the *training* set before using KNN. Precisely, one can preprocess data so that each training feature (column) has a mean of zero and a standard deviation of one. Note that the order is exact. 

In fact, we will see the **effect** of standardizing training and test sets on the predicted values later.

-------

### Install and import libraries

__Notice:__ We have to install some packages for this notebook, and it may takes a few minutes.

In [1]:
# install the packages (note: this may take some time)
install.packages("class")
install.packages("caret")
install.packages("mlbench")
install.packages("e1071")

library(class)
library(caret)
require(mlbench)
library(e1071)
library(base)
require(base)

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
also installing the dependencies ‘generics’, ‘rlang’, ‘recipes’

Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Updating HTML index of packages in '.Library'
Making 'packages.html' ... done
Loading required package: lattice
Loading required package: ggplot2
Loading required package: mlbench


### Step 1- Data collection

For this lesson, we will be using `Sonar` data set (signals) from `mlbench` library. `Sonar` is a system for the detection of objects under water and for measuring the water's depth by emitting sound pulses and detecting. The complete description can be found in [mlbench](https://cran.r-project.org/web/packages/mlbench/mlbench.pdf). For our purposes, this is a two-class (class $R$ and class $M$) classification task with numeric data.

Let's look at the first five rows of `Sonar`

In [2]:
data(Sonar)
head(Sonar)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V52,V53,V54,V55,V56,V57,V58,V59,V60,Class
0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,⋯,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032,R
0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,⋯,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044,R
0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,⋯,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078,R
0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,⋯,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117,R
0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,⋯,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094,R
0.0286,0.0453,0.0277,0.0174,0.0384,0.0990,0.1201,0.1833,0.2105,0.3039,⋯,0.0045,0.0014,0.0038,0.0013,0.0089,0.0057,0.0027,0.0051,0.0062,R


### Step 2- Preparing and exploring the data

It is A data frame with 208 observations on 61 variables, all numerical and one (the Class) nominal.

In [3]:
cat("number of rows and columns are:", nrow(Sonar), ncol(Sonar))

number of rows and columns are: 208 61

Lets check how many $M$ classes and $R$ classes `Sonar` data contain? and check whether `Sonar` data contains any NA in its columns.

In [4]:
base::table(Sonar$Class) 
apply(Sonar, 2, function(x) sum(is.na(x))) 


  M   R 
111  97 

V1    V2    V3    V4    V5    V6    V7    V8    V9   V10   V11   V12   V13 
    0     0     0     0     0     0     0     0     0     0     0     0     0 
  V14   V15   V16   V17   V18   V19   V20   V21   V22   V23   V24   V25   V26 
    0     0     0     0     0     0     0     0     0     0     0     0     0 
  V27   V28   V29   V30   V31   V32   V33   V34   V35   V36   V37   V38   V39 
    0     0     0     0     0     0     0     0     0     0     0     0     0 
  V40   V41   V42   V43   V44   V45   V46   V47   V48   V49   V50   V51   V52 
    0     0     0     0     0     0     0     0     0     0     0     0     0 
  V53   V54   V55   V56   V57   V58   V59   V60 Class 
    0     0     0     0     0     0     0     0     0

Here, we want to manually take samples from our data to split `Sonar` into training and test sets

In [5]:
SEED <- 123
set.seed(SEED)
data <- Sonar[base::sample(nrow(Sonar)), ] # shuffle data first
bound <- floor(0.7 * nrow(data))
df_train <- data[1:bound, ] 
df_test <- data[(bound + 1):nrow(data), ]
cat("number of training and test samples are ", nrow(df_train), nrow(df_test))

number of training and test samples are  145 63

Let's examine if the train and test samples have properly splitted with the almost the same portion of `Class` labels

In [6]:
cat("number of training classes: \n", base::table(df_train$Class)/nrow(df_train))
cat("\n")
cat("number of test classes: \n", base::table(df_test$Class)/nrow(df_test))

number of training classes: 
 0.5310345 0.4689655
number of test classes: 
 0.5396825 0.4603175

To simplify our job, we can create the following data frames

In [7]:
X_train <- subset(df_train, select=-Class)
y_train <- df_train$Class
X_test <- subset(df_test, select=-Class) # exclude Class for prediction
y_test <- df_test$Class

### Step 3 – Training a model on data
Now, we are going to use `knn` function from `class` library with $k=3$

In [8]:
model_knn <- knn(train=X_train,
                 test=X_test,
                 cl=y_train,  # class labels
                 k=3)
model_knn

[1] M M M M R R M M M R M M M R M R R M M M M R M R R M R M R M M R M M M M M M
[39] R R M M M M M R R R R R M M R M R R R R R R M R M
Levels: M R

### Step 4 – Evaluate the model performance
As you can see, `model_knn` with $k=3$ provides the above predictions for the test set `X_test`. Then, we can see how many classes have been correctly or incorrectly classified by comparing to the true labels as follows

In [9]:
conf_mat <- base::table(y_test, model_knn)
conf_mat

      model_knn
y_test  M  R
     M 28  6
     R  8 21

To compute the accuracy, we sum up all the correctly classified observations (located in diagonal) and divide it by the total number of classes

In [10]:
cat("Test accuracy: ", sum(diag(conf_mat))/sum(conf_mat))

Test accuracy:  0.7777778

To assess whether $k=3$ is a good choice and see whether $k=3$ leads to overfitting /underfitting the data, we could use `knn.cv` which does the leave-one-out cross-validations for training set (i.e., it singles out a training sample one at a time and tries to view it as a new example and see what class label it assigns).

Below are the predicted classes for the training set using the leave-one-out cross-validation. Now, let's examine its accuracy

In [11]:
knn_loocv <- knn.cv(train=X_train, cl=y_train, k=3)
knn_loocv

[1] R M R M M R M M M R M R M M M R R R R M M M M M M M R M R M M M M R M R M
 [38] R R R R M R R R R M R R M M M M R R R M M M R R R R M M R M M R R M M M M
 [75] R R M M M R M M M M M R M M M M R R M M R R R R R R M R M M M R M R R M M
[112] M M M R M M M M M R R M M M R M M R M R R M M R R R R M R M M R R M
Levels: M R

Lets create a confusion matrix to compute the accuracy of the training labels `y_train` and the cross-validated predictions `knn_loocv`, same as the above. What can you find from comparing the LOOCV accuracy and the test accuracy above?

In [12]:
conf_mat_cv <- base::table(y_train, knn_loocv)
conf_mat_cv
cat("LOOCV accuracy: ", sum(diag(conf_mat_cv)) / sum(conf_mat_cv))

       knn_loocv
y_train  M  R
      M 67 10
      R 15 53

LOOCV accuracy:  0.8275862

The difference between the cross-validated accuracy and the test accuracy shows that, $k=3$ leads to overfitting. Perhaps we should change $k$ to lessen the overfitting.

### Step 5 – Improve the performance of the model
As noted earlier, we have *not* standardized (as part of preprocessing) our training and test sets. In the rest of the tutorial, we will see the effect of choosing a suitable $k$ through repeated *cross-validations* using `caret` library.

In a *cross-validation* procedure: 
1. The data is divided into the finite number of mutually exclusive subsets 
2. Through each iteration, a subset is set aside, and the remaining subsets are used as the training set
3. The subset that was set aside is used as the test set (prediction)

This is a method of cross-referencing the model built using its own data.

In [13]:
SEED <- 2016
set.seed(SEED)
# create the training data 70% of the overall Sonar data.
in_train <- createDataPartition(Sonar$Class, p=0.7, list=FALSE) # create training indices
ndf_train <- Sonar[in_train, ]
ndf_test <- Sonar[-in_train, ]

Here, we specify the cross-validation method we want to use to find the best $k$ in grid search. Later, we use the built-in `plot` function to assess the changes in accuracy for different choices of $k.$

In [14]:
# lets create a function setup to do 5-fold cross-validation with 2 repeat.
ctrl <- trainControl(method="repeatedcv", number=5, repeats=2)

nn_grid <- expand.grid(k=c(1,3,5,7))
nn_grid

k
1
3
5
7


In [15]:
set.seed(SEED)

best_knn <- train(Class~., data=ndf_train,
                  method="knn",
                  trControl=ctrl, 
                  preProcess = c("center", "scale"),  # standardize
                  tuneGrid=nn_grid)
best_knn

k-Nearest Neighbors 

146 samples
 60 predictor
  2 classes: 'M', 'R' 

Pre-processing: centered (60), scaled (60) 
Resampling: Cross-Validated (5 fold, repeated 2 times) 
Summary of sample sizes: 116, 116, 117, 117, 118, 116, ... 
Resampling results across tuning parameters:

  k  Accuracy   Kappa    
  1  0.8593432  0.7152417
  3  0.8329310  0.6601456
  5  0.7846305  0.5602652
  7  0.7608210  0.5081680

Accuracy was used to select the optimal model using the largest value.
The final value used for the model was k = 1.

So seemingly, $k=1$ has the highest accuracy from repeated cross-validation.

### <span style="color:red">(Optional) Exercise</span>

Try to do dimensionality reduction as part of preprocess to achieve *higher* testing accuracy than above. This may not have a definite solution and it depends on how hard you try!

If you are going to use `caret`, [here](https://github.com/topepo/caret/blob/master/pkg/caret/R/preProcess.R) are the available preprocess options with [explanations](http://topepo.github.io/caret/preprocess.html#pp).

Use the above `best_knn` to make `predictions` on the test set (remeber to remove the `Class` for prediction). Then create the much better version of confusion matrix with `confusionMatrix` function from `caret` and examine the accuracy and its $\%95$ confidence interval.

In fact, the above result indicates $k=1$ (as could be guessed) is also overfitting, though it might be a better option than $k=3.$ Since the initial dimension of our data is high ($61$ is considered high!), then you might have suspected the better approach, as we said at the beginning of tutorial, is to preform *dimensionality reduction* as part of preprocessing.

### <span style="color:red">(Optional) Exercise</span>

Try to do dimensionality reduction as part of preprocess to achieve *higher* testing accuracy than above. This may not have a definite solution and it depends on how hard you try!

If you are going to use `caret`, [here](https://github.com/topepo/caret/blob/master/pkg/caret/R/preProcess.R) are the available preprocess options with [explanations](http://topepo.github.io/caret/preprocess.html#pp).

In [16]:
SEED <- 123 
set.seed(SEED) 
ctrl <- trainControl(method="repeatedcv", number=5, repeats=5) 
nn_grid <- expand.grid(k=c(1, 3, 5, 7)) 
best_knn_reduced <- train( Class~., data=ndf_train, method="knn", 
                            trControl=ctrl, preProcess=c("center", "scale","YeoJohnson"))
X_test <- subset(ndf_test, select=-Class) 
pred_reduced <- predict(best_knn_reduced, newdata=X_test, model="best") 
conf_mat_best_reduced <- confusionMatrix(ndf_test$Class, pred_reduced) 
conf_mat_best_reduced 


Confusion Matrix and Statistics

          Reference
Prediction  M  R
         M 29  4
         R  9 20
                                          
               Accuracy : 0.7903          
                 95% CI : (0.6682, 0.8834)
    No Information Rate : 0.6129          
    P-Value [Acc > NIR] : 0.002277        
                                          
                  Kappa : 0.5744          
 Mcnemar's Test P-Value : 0.267257        
                                          
            Sensitivity : 0.7632          
            Specificity : 0.8333          
         Pos Pred Value : 0.8788          
         Neg Pred Value : 0.6897          
             Prevalence : 0.6129          
         Detection Rate : 0.4677          
   Detection Prevalence : 0.5323          
      Balanced Accuracy : 0.7982          
                                          
       'Positive' Class : M               
                                          

-----------------

## Want to learn more?

IBM SPSS Modeler is a comprehensive analytics platform that has many machine learning algorithms. It has been designed to bring predictive intelligence to decisions made by individuals, by groups, by systems – by your enterprise as a whole. A free trial is available through this course, available here: [SPSS Modeler for Mac users](https://cocl.us/ML0151EN_SPSSMod_mac) and [SPSS Modeler for Windows users](https://cocl.us/ML0151EN_SPSSMod_win)

Also, you can use Data Science Experience to run these notebooks faster with bigger datasets. Data Science Experience is IBM's leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, DSX enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of DSX users today with a free account at [Data Science Experience](https://cocl.us/ML0151EN_DSX)

### Thanks for completing this lesson!

Notebook created by: [Ehsan M. Kermani](https://www.linkedin.com/in/ehsanmkermani)

### References:

* [K-nearest neighbors algorithm](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm)
* [Short introdunction to caret](https://cran.r-project.org/web/packages/caret/vignettes/caret.pdf)
* [Predictive modeling with caret](https://www.r-project.org/nosvn/conferences/useR-2013/Tutorials/kuhn/user_caret_2up.pdf)

<hr>
Copyright &copy; 2016 [Cognitive Class](https://cognitiveClass.ai/?utm_source=bducopyrightlink&utm_medium=dswb&utm_campaign=bdu). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).